In [22]:
# Libraries
import numpy as np   
import time
import scipy.sparse.linalg as spy
import warnings
import matplotlib.pyplot as plt
%matplotlib inline 

warnings.filterwarnings('ignore')


def pos_def(n,x=5):
    A = np.random.rand(n,n)
    return (A+A.T) + x*np.eye(n)

#n = 10

#for i in range(100):
#    A = pos_def(n, x = 8)
#    k = np.linalg.cond(A)
#    if k > 100 and (np.linalg.eig(A)[0] > 0).all():
#        print "Matrix selected"
#        print "Condition number: ", k
#        break
        
#x_sol = np.floor(np.random.rand(n) * 100)
#b = np.dot(A, x_sol)

In [17]:
# A better implementation of preconditioned GMRes without Cauchy integral (left)
def prec_gmres(A_0,b_0,alpha,tol=1e-6,left = True):
    it = b_0.shape[0]
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    # copy
    A = np.copy(A_0)
    b = np.copy(b_0)
    M = A + alpha*np.identity(b.shape[0])
    
    # Left preconditioner -> A and b changes
    if left:
        b = gmres(M,b,tol=tol)[0]
    
    r = b - np.dot(A,x0)
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if left:
            w = gmres(M, np.dot(A,Q[:,i]),tol=tol)[0]
        else:
            w = np.dot(A, gmres(M, Q[:,i],tol=tol)[0])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left:
        return x_tild, i+1
    else:
        return gmres(M, x_tild,tol=tol)[0],i+1

In [18]:
def trapezoid2(myfun, N, a, b):
    x = np.linspace(0, b, N/2) # We want N bins, so N+1 points  
    h = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0
    for i in xmiddle:
        int_val += 2*myfun(i).real
    int_val = 2*myfun(a).real + 2*int_val + 2*myfun(0)# + myfun(b)
    return 0.5*h*int_val

def z(t, c, r):
    return c + r*np.complex(np.cos(t), np.sin(t))

def dz(t, r):
    return r*np.complex(np.cos(t), np.sin(t))

def g(t,l,L,alpha,v,f, tol=1e-6):
    centro = (l + L)/2.
    radio = (L - l)/2.
    fz = f(z(t, centro, radio))
    dzz = dz(t, radio)
    p = fz*dzz
    # Separamos las matrices
    M = (centro + dzz - alpha)*np.identity(v.shape[0]) - A
    a = M.real
    b = M.imag
    al = v.real
    bet = v.imag
    
    # Construímos el nuevo sistema Hx = r
    H = np.zeros((a.shape[0]*2, a.shape[1]*2))
    H[:a.shape[0],:a.shape[1]] = a
    H[a.shape[0]:,a.shape[1]:] = a
    H[:a.shape[0],a.shape[1]:] = -b
    H[a.shape[0]:,:a.shape[1]] = b
    r = np.zeros((al.shape[0]*2))
    r[:al.shape[0]] = al
    r[al.shape[0]:] = bet
    
    sol = gmres(H,r,tol=tol)[0]
    gmr = sol[:(sol.shape[0]/2)] + 1j*sol[(sol.shape[0]/2):]
    #gmr = p*spy.gmres((centro + dzz - alpha)*np.identity(v.shape[0]) - A, v)[0]
    return p*gmr


def cauchy_integral(l, L, alpha, v, Nf = 50,N=128,tol=1e-6):
    #f = lambda x: (1. - (alpha/x)**(Nf+1))/(x - alpha)
    # Nueva función
    f = lambda x: (1. - (alpha/(2*x))**(Nf+1))/(x - alpha/2.)
    g1 = lambda t: g(t,l,L,alpha,v,f,tol=tol)
    val = trapezoid2(g1, N, -np.pi, np.pi) / (2.*np.pi)
    return val

In [19]:
# GMRes using contour integral to compute the preconditioner
def gmres(A_0, b_0, tol=1e-6, prec=False, left=True, alpha=0.0,aux_tol=1e-6):
    it = b_0.shape[0]
    Q = np.zeros((b_0.shape[0], it+1))
    H = np.zeros((it+1,it))
    x0 = np.zeros((b_0.shape[0]))
    
    A = np.copy(A_0)
    b = np.copy(b_0)
    
    if prec:
        l = alpha
        L = np.amax(np.sum(np.abs(A),axis=1))
        if left:
            b = cauchy_integral(l, L, alpha, b,tol=aux_tol)
    
    r = b 
    beta0 = np.linalg.norm(b)
    beta1 = np.linalg.norm(r)
    Q[:,0] = r/beta1
    
    for i in range(it):
        e = np.zeros((i+2))
        e[0] = 1
        
        if prec:
            if left:
                w = cauchy_integral(l,L,alpha,np.dot(A,Q[:,i]),tol=aux_tol)
            else:
                w = np.dot(A, cauchy_integral(l, L, alpha, Q[:,i],tol=aux_tol))
        else:
            w = np.dot(A,Q[:,i])
        
        for j in range(i+1):
            h = np.dot(Q[:,j],w)
            w -= h*Q[:,j]
            H[j,i] = h
        
        H[i+1,i] = np.linalg.norm(w)
        
        if H[i+1,i] != 0:
            Q[:,i+1] = w/H[i+1,i]
        
        y,_,_,_ = np.linalg.lstsq(H[:i+2,:i+1], beta1*e)
        residual = np.linalg.norm(np.dot(H[:i+2,:i+1],y) - beta1*e)
        
        if H[i+1,i] == 0 or residual/beta0 < tol:
            break
    
    x_tild = np.dot(Q[:,:i+1], y)
    if left or not prec:
        return x_tild,i+1
    else:
        return cauchy_integral(l, L, alpha, x_tild,tol=aux_tol),i+1

In [20]:
alpha = 0.05
print "Exact solution:\n", x_sol
print "\nNp Solve:\n", spy.gmres(A,b)
print "\nNormal GMRes:\n", gmres(A,b)
print "\nLeft Prec NC:\n", prec_gmres(A,b,alpha)
#print "\nRight Prec NC:\n", prec_gmres(A,b,alpha,left=False)
print "\nCauchy Left Prec:\n", gmres(A, b,prec=True,alpha=alpha)
#print "\nCauchy Right Prec:\n", gmres(A,b,prec=True,left=False,alpha=alpha)

Exact solution:
[ 52.  72.  36.  56.  77.  85.   8.  32.  66.  89.]

Np Solve:
(array([ 52.00013256,  72.00019885,  36.00016594,  56.00002952,
        77.00009583,  84.99985865,   7.99998938,  31.99985327,
        65.99979692,  88.99993233]), 0)

Normal GMRes:
(array([ 52.00013256,  72.00019885,  36.00016594,  56.00002952,
        77.00009583,  84.99985865,   7.99998938,  31.99985327,
        65.99979692,  88.99993233]), 6)

Left Prec NC:
(array([ 52.00005805,  72.00007238,  35.99992934,  56.00001676,
        76.9999342 ,  85.00001445,   7.99991736,  32.00003854,
        65.99999392,  88.99996676]), 2)

Cauchy Left Prec:
(array([ 52.00001471,  72.00001839,  35.99998233,  56.00000425,
        76.99998353,  85.00000354,   7.99997915,  32.00000953,
        65.99999827,  88.99999159]), 2)


In [15]:
all_h = []

In [26]:
def graphics(n):
    it_1 = []
    it_2 = []
    it_3 = []
    for i in range(10,n+10,10):
        A = pos_def(i)
        x_sol = np.floor(np.random.rand(i) * 100)
        b = np.dot(A, x_sol)
        
        print A.shape, b.shape, x_sol.shape
        it_1.append(gmres(A,b)[1]) 
        it_2.append(prec_gmres(A,b,alpha)[1])
        it_3.append(gmres(A, b,prec=True,alpha=alpha)[1])
        
        
    normal_line = plt.plot(it_1, 'r-^', label="Gmres")
    prec_line = plt.plot(it_2, 'b-^', label="Prec_Gmres")
    cauchy_line = plt.plot(it_3, 'g-^', label="Cauchy Gmres")
    plt.legend()
    plt.xlabel('Matrix dimension')
    plt.ylabel('Iterations')
    plt.title('Iterations comparison')
    plt.show()     
        
    return

graphics(100)


(10, 10) (10,) (10,)
(20, 20) (20,) (20,)


ValueError: operands could not be broadcast together with shapes (20,20) (10,10) 